In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
warnings.filterwarnings("ignore")

df=pd.read_csv("/kaggle/input/health-and-sleep-statistics/Health_Sleep_Statistics.csv")
print(df.info())
print(df.isnull().sum())
print(df.head())

# **EDA**

In [ ]:
print(df.describe())


In [ ]:
sns.histplot(df['Age'], bins=10, kde=True)
plt.title('Distribution of Age')
plt.show()


In [ ]:
sns.countplot(x='Gender', data=df)
plt.title('Gender Distribution')
plt.show()


In [ ]:
sns.histplot(df['Sleep Quality'], bins=10, kde=True)
plt.title('Distribution of Sleep Quality')
plt.show()


In [ ]:
df['Bedtime'] = pd.to_datetime(df['Bedtime'], format='%H:%M')
df['Bedtime Hour'] = df['Bedtime'].dt.hour + df['Bedtime'].dt.minute / 60
sns.histplot(df['Bedtime Hour'], bins=24, kde=True)
plt.title('Distribution of Bedtime')
plt.xlabel('Hour of the Day')
plt.show()


In [ ]:
df['Wake-up Time'] = pd.to_datetime(df['Wake-up Time'], format='%H:%M')
df['Wake-up Hour'] = df['Wake-up Time'].dt.hour + df['Wake-up Time'].dt.minute / 60
sns.histplot(df['Wake-up Hour'], bins=24, kde=True)
plt.title('Distribution of Wake-up Time')
plt.xlabel('Hour of the Day')
plt.show()


In [ ]:
sns.histplot(df['Daily Steps'], bins=10, kde=True)
plt.title('Distribution of Daily Steps')
plt.show()


In [ ]:
sns.histplot(df['Calories Burned'], bins=10, kde=True)
plt.title('Distribution of Calories Burned')
plt.show()


In [ ]:
sns.countplot(x='Physical Activity Level', data=df)
plt.title('Physical Activity Level Distribution')
plt.show()


In [ ]:
sns.countplot(x='Dietary Habits', data=df)
plt.title('Dietary Habits Distribution')
plt.show()


In [ ]:
sns.countplot(x='Sleep Disorders', data=df)
plt.title('Sleep Disorders Distribution')
plt.show()


In [ ]:
sns.countplot(x='Medication Usage', data=df)
plt.title('Medication Usage Distribution')
plt.show()


In [ ]:
correlation_matrix = df[['Age', 'Sleep Quality', 'Daily Steps', 'Calories Burned']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
sns.scatterplot(x='Age', y='Sleep Quality', data=df)
plt.title('Sleep Quality vs. Age')
plt.show()


In [ ]:
sns.scatterplot(x='Daily Steps', y='Sleep Quality', data=df)
plt.title('Sleep Quality vs. Daily Steps')
plt.show()


In [ ]:
sns.scatterplot(x='Calories Burned', y='Sleep Quality', data=df)
plt.title('Sleep Quality vs. Calories Burned')
plt.show()


In [ ]:
sns.boxplot(x='Gender', y='Sleep Quality', data=df)
plt.title('Sleep Quality by Gender')
plt.show()


In [ ]:
sns.boxplot(x='Physical Activity Level', y='Sleep Quality', data=df)
plt.title('Sleep Quality by Physical Activity Level')
plt.show()


In [ ]:
sns.boxplot(x='Dietary Habits', y='Sleep Quality', data=df)
plt.title('Sleep Quality by Dietary Habits')
plt.show()


In [ ]:
sns.boxplot(x='Medication Usage', y='Sleep Quality', data=df)
plt.title('Sleep Quality by Medication Usage')
plt.show()


# **Model**

In [ ]:
categorical_features = ['Gender', 'Physical Activity Level', 'Dietary Habits', 'Sleep Disorders', 'Medication Usage']
numeric_features = ['Age', 'Daily Steps', 'Calories Burned']

X = df[categorical_features + numeric_features]
y = df['Sleep Quality']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', 'passthrough', numeric_features)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

print("Model R^2 Score: ", model.score(X_test, y_test))

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")


In [ ]:
residuals = y_test - y_pred

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')

plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted values')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')

plt.tight_layout()
plt.show()


In [ ]:
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f'Cross-Validation R² Scores: {cv_scores}')
print(f'Average Cross-Validation R² Score: {cv_scores.mean()}')
